In [ ]:
import geopandas as gpd
import pandas as pd

In [ ]:
data = pd.read_parquet("/data/processed_data/Bevoelkerung100M.parquet")

In [ ]:
data

In [ ]:
gdf = gpd.GeoDataFrame(
    data, geometry=gpd.points_from_xy(data[("", "E")], data[("", "N")]), crs="EPSG:3035"
)
gdf = gdf.rename(columns={"": "geometry"})
gdf = gdf.drop(["N", "E"], axis=1, level=1)

In [ ]:
# Buffer the points using a square cap style
# Note cap_style: round = 1, flat = 2, square = 3
gdf["geometry"] = gdf["geometry"].buffer(50, cap_style=3)

In [ ]:
gdf

In [ ]:
gdf.columns.get_level_values(0)

# Nationality

In [ ]:
ntn = gdf[["NATIONALITY", "geometry"]]

In [ ]:
ntn.columns = ntn[["NATIONALITY", "geometry"]].columns.get_level_values(1)

In [ ]:
ntn

In [ ]:
ntn["% German"] = ntn["Germany"] / (ntn["Germany"].fillna(0) + ntn["Abroad"].fillna(0))

In [ ]:
fbg_ntn = ntn.cx[4150000:4170000, 2760000:2780000]

In [ ]:
fbg_ntn.explore(column="% German")

# Religion 

In [ ]:
gdf.RELIGION

In [ ]:
rgn = gdf[["RELIGION", "geometry"]]
rgn.columns = rgn[["RELIGION", "geometry"]].columns.get_level_values(1)

In [ ]:
rgn

In [ ]:
rgn["Roman Catholic Church (public law)"].sum()

In [ ]:
rgn["Other, none, no information"].sum()

In [ ]:
rgn["Evangelical Church (public law)"].sum()

In [ ]:
rgn["Total"] = (
    rgn["Roman Catholic Church (public law)"].fillna(0)
    + rgn["Other, none, no information"].fillna(0)
    + rgn["Evangelical Church (public law)"].fillna(0)
)

In [ ]:
rgn["% Roman"] = rgn["Roman Catholic Church (public law)"] / rgn["Total"]
rgn["% Evangelical"] = rgn["Evangelical Church (public law)"] / rgn["Total"]
rgn["% Other"] = rgn["Other, none, no information"] / rgn["Total"]

In [ ]:
rgn

In [ ]:
fbg_rgn = rgn.cx[4150000:4170000, 2760000:2780000]

In [ ]:
fbg_rgn.explore(column="% Roman")

In [ ]:
fbg_rgn.explore(column="% Evangelical")

In [ ]:
fbg_rgn.explore(column="% Other")

# Age

In [ ]:
age = gdf[["AGE_10", "geometry"]]
age.columns = age[["AGE_10", "geometry"]].columns.get_level_values(1)

In [ ]:
age.columns

In [ ]:
age["Total"] = age[
    [
        "Under 10",
        "20 - 29",
        "60 - 69",
        "10 - 19",
        "40 - 49",
        "50 - 59",
        "30 - 39",
        "70 - 79",
        "80 and older",
    ]
].sum(axis=1)

In [ ]:
age["Average"] = (
    age["Under 10"].fillna(0) * 5
    + age["10 - 19"].fillna(0) * 15
    + age["20 - 29"].fillna(0) * 25
    + age["30 - 39"].fillna(0) * 35
    + age["40 - 49"].fillna(0) * 45
    + age["50 - 59"].fillna(0) * 55
    + age["60 - 69"].fillna(0) * 65
    + age["70 - 79"].fillna(0) * 75
    + age["80 and older"].fillna(0) * 85
) / age["Total"]

In [ ]:
age

In [ ]:
fbg_age = age.cx[4150000:4170000, 2760000:2780000]

In [ ]:
fbg_age.explore(column="Average")